In [70]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 有效广告及其特征

In [71]:
import pandas as pd
from pathlib import Path
import os, re
from functools import reduce, partial
from utils import or_func
from tqdm import trange

In [72]:
root = Path('../data')
root
os.listdir(root)

PosixPath('../data')

['2019-03-10_log_exposed.csv',
 '2019-02-23_log.csv',
 '2019-02-28_log.csv',
 '2019-03-14_log_exposed.csv',
 '2019-02-22_log_exposed.csv',
 'log_label.tgz',
 '2019-03-16_log.csv',
 '2019-03-11_log_exposed.csv',
 '2019-02-16_log_exposed.csv',
 '2019-03-12_log_exposed.csv',
 '2019-02-28_log_exposed.csv',
 '2019-03-06_log.csv',
 '2019-03-13_log.csv',
 '2019-02-23_log_exposed.csv',
 '2019-03-19_log.csv',
 '2019-03-01_log_exposed.csv',
 '2019-02-26_log_exposed.csv',
 '2019-03-01_log.csv',
 '2019-02-20_log_exposed.csv',
 '2019-02-19_log_exposed.csv',
 '2019-02-24_log.csv',
 '2019-03-03_log.csv',
 '2019-02-27_log_exposed.csv',
 '2019-03-03_log_exposed.csv',
 '2019-02-17_log.csv',
 '2019-02-18_log.csv',
 '2019-03-05_log_exposed.csv',
 '2019-03-07_log_exposed.csv',
 '2019-02-16_log.csv',
 '2019-02-26_log.csv',
 '.ipynb_checkpoints',
 '2019-03-09_log.csv',
 '2019-03-15_log.csv',
 '2019-02-21_log.csv',
 '2019-03-11_log.csv',
 'test_set.csv',
 'users_data.csv',
 '2019-03-07_log.csv',
 '2019-03-13_

In [73]:
ads = pd.read_csv(root/'ad_static_operation.csv', encoding='utf-8')
ads.head()
ads.info()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,2,90
1,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,3,all
2,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-17 00:00:00,1,1,0
4,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-18 23:38:55,1,2,90


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613409 entries, 0 to 613408
Data columns (total 11 columns):
广告id       613409 non-null int64
创建时间       613409 non-null object
广告账户id     613409 non-null int64
商品id       613409 non-null int64
商品类型       613409 non-null int64
广告行业id     613409 non-null int64
素材尺寸       613409 non-null int64
创建/修改时间    613409 non-null object
操作类型       613409 non-null int64
修改字段       613409 non-null int64
操作后的字段值    613409 non-null object
dtypes: int64(8), object(3)
memory usage: 51.5+ MB


- 操作后的字段值:
    - 广告状态取值:1- 正常,0-失效
    - 出价:整数(单位分)
    - 投放时段:字符串
    - 人群定向:字符串。格式如下: feature_name1:feature_value1,feature_value2|feature_name2:feature_value3,feature_value4|...
        此处 feature_name 取值同用户属性文件中的各列属性名,feature_value 取值 id 同用户属性文件中的定义,不同feature 用“|”分隔,不同 feature 取值用逗号分隔。
      广告通过人群定向
的设置来召回对应的用户请求,对应的人群规则:不同 feature_name 是
求 交 集 , 同 一 featurename 下 不 同 的 value 求 并 集 , 未 定 义 的
feature_name 则 表 示 该 维 度 不 限 。 举 例 如 : 定 向 设 置 为
age:51,62,73,84|gender:1|area:1,3,5 ; 则表示该广告能被 “
(年龄 id 为
51 或 62 或 73 或 84) 且 (性别取值为 1) 且 (地域取值为 1 或 3
或 5) ”的用户召回(即在这些用户上有曝光机会)

In [74]:
users = pd.read_csv(root/'users_data.csv', encoding='utf-8')
users.head()
users.info()

,用户id,Age,Gender,地域,婚恋状态,学历,消费能力,设备,工作状态,连接类型,行为兴趣
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396718 entries, 0 to 1396717
Data columns (total 11 columns):
用户id      1396718 non-null int64
Age       1396718 non-null int64
Gender    1396718 non-null int64
地域        1396718 non-null object
婚恋状态      1396718 non-null object
学历        1396718 non-null int64
消费能力      1396718 non-null int64
设备        1396718 non-null int64
工作状态      1396718 non-null object
连接类型      1396718 non-null int64
行为兴趣      1396718 non-null object
dtypes: int64(7), object(4)
memory usage: 117.2+ MB


### 多值特征的处理
- 离散为 bool 特征

In [75]:
users.shape

(1396718, 11)

## 广告操作数据

In [76]:
ads.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
613404,209095,2019-03-19 13:21:51,25817,31956,18,198,64,2019-03-19 13:21:51,2,4,"281474976710655,281474976710655,28147497671065..."
613405,209095,2019-03-19 13:21:51,25817,31956,18,198,64,2019-03-19 21:23:09,1,2,20
613406,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,2,20
613407,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,3,"behavior:22855,23131"
613408,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,4,"281474976710655,281474976710655,28147497671065..."


In [77]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
cols1 = ['state', 'price', 'time', 'people']

In [78]:
new_ads = ads
new_ads = new_ads.reindex(columns=new_ads.columns.to_list() + cols + cols1)

In [79]:
new_ads.head(7).T

,0,1,2,3,4,5,6
广告id,593323,593323,593323,593323,593323,593323,593323
创建时间,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55
广告账户id,26889,26889,26889,26889,26889,26889,26889
商品id,-1,-1,-1,-1,-1,-1,-1
商品类型,3,3,3,3,3,3,3
广告行业id,85,85,85,85,85,85,85
素材尺寸,64,64,64,64,64,64,64
创建/修改时间,2015-04-07 09:43:55,2015-04-07 09:43:55,2015-04-07 09:43:55,2019-02-17 00:00:00,2019-02-18 23:38:55,2019-02-18 00:00:00,2019-03-01 00:00:00
操作类型,2,2,2,1,1,1,1
修改字段,2,3,4,1,2,1,1


- 操作类型:1-修改,2-新建
- 修改字段:1-广告状态,2-出价,3-人群定向,4-广告时段设置

In [80]:
ops_field = {1: 'state', 2: 'price', 3: 'people', 4: 'time'}

In [81]:
new_ads.groupby(by=['操作类型', '修改字段']).min()

广告id                 创建时间  广告账户id  商品id  商品类型  广告行业id  素材尺寸  \
操作类型 修改字段                                                                
1    1       31  2015-04-07 09:43:55       6    -1     1      -1     1   
     2       32  2015-04-07 09:43:55      16    -1     1      -1     1   
     3      764  2017-08-07 07:51:44      80    -1     1       1     1   
     4      235  2017-06-18 10:47:25      34    -1     1      -1     1   
2    2       31  2015-04-07 09:43:55       6    -1     1      -1     1   
     3       31  2015-04-07 09:43:55       6    -1     1      -1     1   
     4       31  2015-04-07 09:43:55       6    -1     1      -1     1   

                       创建/修改时间  \
操作类型 修改字段                        
1    1     2017-06-06 01:44:47   
     2     2019-02-17 02:14:17   
     3     2019-02-17 02:14:17   
     4     2019-02-17 02:14:28   
2    2     2015-04-07 09:43:55   
     3     2015-04-07 09:43:55   
     4     2015-04-07 09:43:55   

                                                     操作后的字段值  age  ...  \
操作类型 修改字段                                                          ...   
1    1                                                     0  NaN  ...   
     2                                                     1  NaN  ...   
     3     age:1,217,731,739,333,393,988,366,608,942,638|...  NaN  ...   
     4                                         0,0,0,0,0,0,0  NaN  ...   
2    2                                                    10  NaN  ...   
     3     age:1,217,731,739,333,393,988,366,608,942,638|...  NaN  ...   
     4                                0,0,0,0,0,0,1073741823  NaN  ...   

           consuptionAbility  device  education  gender  status  work  state  \
操作类型 修改字段                                                                      
1    1                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     2                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     3                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     4                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
2    2                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     3                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   
     4                   NaN     NaN        NaN     NaN     NaN   NaN    NaN   

           price  time  people  
操作类型 修改字段                       
1    1       NaN   NaN     NaN  
     2       NaN   NaN     NaN  
     3       NaN   NaN     NaN  
     4       NaN   NaN     NaN  
2    2       NaN   NaN     NaN  
     3       NaN   NaN     NaN  
     4       NaN   NaN     NaN  

[7 rows x 23 columns]

In [82]:
new_ads.groupby(by=['操作类型']).count()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,修改字段,操作后的字段值,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
操作类型,,,,,,,,,,,,,,,,,,,,,
1,527063,527063,527063,527063,527063,527063,527063,527063,527063,527063,...,0,0,0,0,0,0,0,0,0,0
2,86346,86346,86346,86346,86346,86346,86346,86346,86346,86346,...,0,0,0,0,0,0,0,0,0,0


- 按照操作类型排序，降表格分离开
- 527434 是修改 86748 是新建

In [83]:
new_ads.sort_values(by='操作类型', inplace=True)
new_ads.reset_index(drop=True, inplace=True)

In [84]:
new_ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
0,524388,2019-02-12 21:34:29,25252,26229,13,17,34,2019-02-27 05:37:19,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,337561,2019-02-19 10:07:05,7955,-1,1,230,64,2019-02-26 22:27:50,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,337561,2019-02-19 10:07:05,7955,-1,1,230,64,2019-02-26 22:27:46,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,337561,2019-02-19 10:07:05,7955,-1,1,230,64,2019-02-26 22:10:36,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,337561,2019-02-19 10:07:05,7955,-1,1,230,64,2019-02-24 00:00:00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
new_ads.loc[527430:527438, :]

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527430,556441,2019-03-08 08:48:16,24640,-1,1,230,40,2019-03-08 08:48:16,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527431,434439,2019-03-14 04:42:29,29298,32811,13,21,30,2019-03-14 04:42:29,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527432,595335,2019-03-18 12:59:29,16395,-1,1,156,64,2019-03-18 12:59:29,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527433,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527434,281828,2019-03-18 12:53:27,19109,-1,1,21,64,2019-03-18 12:53:27,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527435,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527436,470460,2019-03-17 01:26:44,14756,-1,1,25,64,2019-03-17 01:26:44,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527437,215012,2019-03-08 08:44:25,13464,-1,1,90,30,2019-03-08 08:44:25,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527438,532947,2019-03-09 21:39:52,18373,22316,13,12,34,2019-03-09 21:39:52,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
new_ads_fix = new_ads.loc[:527433, :]  # 包括边界
new_ads_crt = new_ads.loc[527434:, :]

In [87]:
new_ads_fix.tail()
new_ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527429,38068,2019-03-12 08:52:33,29084,23132,13,122,30,2019-03-12 08:52:33,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527430,556441,2019-03-08 08:48:16,24640,-1,1,230,40,2019-03-08 08:48:16,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527431,434439,2019-03-14 04:42:29,29298,32811,13,21,30,2019-03-14 04:42:29,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527432,595335,2019-03-18 12:59:29,16395,-1,1,156,64,2019-03-18 12:59:29,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527433,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527434,281828,2019-03-18 12:53:27,19109,-1,1,21,64,2019-03-18 12:53:27,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527435,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527436,470460,2019-03-17 01:26:44,14756,-1,1,25,64,2019-03-17 01:26:44,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527437,215012,2019-03-08 08:44:25,13464,-1,1,90,30,2019-03-08 08:44:25,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527438,532947,2019-03-09 21:39:52,18373,22316,13,12,34,2019-03-09 21:39:52,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 修改操作

In [88]:
new_ads_fix.fillna(value=-999, inplace=True)
new_ads_crt.fillna(value=-999, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [89]:
new_ads_fix.tail()
new_ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527429,38068,2019-03-12 08:52:33,29084,23132,13,122,30,2019-03-12 08:52:33,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527430,556441,2019-03-08 08:48:16,24640,-1,1,230,40,2019-03-08 08:48:16,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527431,434439,2019-03-14 04:42:29,29298,32811,13,21,30,2019-03-14 04:42:29,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527432,595335,2019-03-18 12:59:29,16395,-1,1,156,64,2019-03-18 12:59:29,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527433,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
527434,281828,2019-03-18 12:53:27,19109,-1,1,21,64,2019-03-18 12:53:27,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527435,324599,2019-03-15 14:02:11,16913,5349,13,224,30,2019-03-15 14:02:11,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527436,470460,2019-03-17 01:26:44,14756,-1,1,25,64,2019-03-17 01:26:44,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527437,215012,2019-03-08 08:44:25,13464,-1,1,90,30,2019-03-08 08:44:25,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
527438,532947,2019-03-09 21:39:52,18373,22316,13,12,34,2019-03-09 21:39:52,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [90]:
ops_field

{1: 'state', 2: 'price', 3: 'people', 4: 'time'}

In [91]:
new_ads_fix.groupby(by='广告id').count().head(10)

,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
广告id,,,,,,,,,,,,,,,,,,,,,
31,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
32,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66
84,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
85,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
128,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
139,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
168,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
180,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
193,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [92]:
new_ads_fix.sort_values(by='创建时间').head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
357268,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-03-01 00:00:00,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
357269,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-18 00:00:00,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
357270,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-18 23:38:55,1,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
357271,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-17 00:00:00,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
357263,146563,2015-06-04 09:11:32,23511,-1,3,118,64,2019-03-02 00:00:00,1,1,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [93]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,2,90
1,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,3,all
2,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-17 00:00:00,1,1,0
4,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-18 23:38:55,1,2,90


In [94]:
ad_static = pd.read_csv(root/'ad_static.csv', low_memory=False, encoding='utf-8')

In [95]:
ad_static.sort_values(by='创建时间').head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64
1,146563,2015-06-04 09:11:32,23511,-1,3,118,64
2,216969,2016-03-10 06:35:34,26970,-1,1,33,1
3,357401,2016-03-10 06:35:35,26970,-1,1,33,1
4,542734,2016-03-10 06:35:35,26970,-1,1,33,34


In [96]:
new_ads_fix.groupby(by='广告id').count().shape

(29285, 24)

In [97]:
ads.sort_values('创建时间', inplace=True)
ads.reset_index(drop=True, inplace=True)

In [98]:
ads.shape

(613409, 11)

In [99]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,2,90
1,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,3,all
2,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2015-04-07 09:43:55,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-17 00:00:00,1,1,0
4,593323,2015-04-07 09:43:55,26889,-1,3,85,64,2019-02-18 23:38:55,1,2,90


- 创建操作是在同一个广告上进行叠加

In [100]:
new_ads_crt[new_ads_crt['广告id']==85]

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
587475,85,2019-02-27 07:52:07,6944,11250,13,170,34,2019-02-27 07:52:07,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
587476,85,2019-02-27 07:52:07,6944,11250,13,170,34,2019-02-27 07:52:07,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
587506,85,2019-02-27 07:52:07,6944,11250,13,170,34,2019-02-27 07:52:07,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [101]:
new_ads_crt.sort_values('广告id', inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [102]:
new_ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
545737,31,2019-03-12 23:22:50,12577,18683,13,224,40,2019-03-12 23:22:50,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
545727,31,2019-03-12 23:22:50,12577,18683,13,224,40,2019-03-12 23:22:50,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
545726,31,2019-03-12 23:22:50,12577,18683,13,224,40,2019-03-12 23:22:50,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
547206,32,2018-11-30 07:40:17,18752,32534,13,136,40,2018-11-30 07:40:17,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
547110,32,2018-11-30 07:40:17,18752,32534,13,136,40,2018-11-30 07:40:17,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [103]:
v_crt = new_ads_crt.groupby('广告id', sort=True).count()['广告账户id'].to_list()
v_crt[:5]

[3, 3, 3, 3, 3]

In [104]:
v_id = new_ads_crt.groupby('广告id', sort=True).count().index.to_list()
v_id[:5]

[31, 32, 84, 85, 128]

In [105]:
new_ads_crt.shape
len(v_crt)

(85975, 25)

28685

- 创建新列表保存--待优化

In [106]:
ads_crt = pd.DataFrame(columns=new_ads_crt.columns)
ads_crt['广告id'] = v_id
ads_crt.head()
ads_crt.shape

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
0,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(28685, 25)

In [107]:
lens = len(ads_crt)

In [108]:
new_ads_crt.reset_index(drop=True, inplace=True)

In [109]:
new_ads_crt.head().T

,0,1,2,3,4
广告id,31,31,31,32,32
创建时间,2019-03-12 23:22:50,2019-03-12 23:22:50,2019-03-12 23:22:50,2018-11-30 07:40:17,2018-11-30 07:40:17
广告账户id,12577,12577,12577,18752,18752
商品id,18683,18683,18683,32534,32534
商品类型,13,13,13,13,13
广告行业id,224,224,224,136,136
素材尺寸,40,40,40,40,40
创建/修改时间,2019-03-12 23:22:50,2019-03-12 23:22:50,2019-03-12 23:22:50,2018-11-30 07:40:17,2018-11-30 07:40:17
操作类型,2,2,2,2,2
修改字段,2,3,4,4,2


In [110]:
dfg = new_ads_crt.iloc[3:6,:]
len(dfg)
dfg

3

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,...,consuptionAbility,device,education,gender,status,work,state,price,time,people
3,32,2018-11-30 07:40:17,18752,32534,13,136,40,2018-11-30 07:40:17,2,4,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,32,2018-11-30 07:40:17,18752,32534,13,136,40,2018-11-30 07:40:17,2,2,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
5,32,2018-11-30 07:40:17,18752,32534,13,136,40,2018-11-30 07:40:17,2,3,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [111]:
list(enumerate(dfg.columns))

[(0, '广告id'),
 (1, '创建时间'),
 (2, '广告账户id'),
 (3, '商品id'),
 (4, '商品类型'),
 (5, '广告行业id'),
 (6, '素材尺寸'),
 (7, '创建/修改时间'),
 (8, '操作类型'),
 (9, '修改字段'),
 (10, '操作后的字段值'),
 (11, 'age'),
 (12, 'area'),
 (13, 'behavior'),
 (14, 'connectionType'),
 (15, 'consuptionAbility'),
 (16, 'device'),
 (17, 'education'),
 (18, 'gender'),
 (19, 'status'),
 (20, 'work'),
 (21, 'state'),
 (22, 'price'),
 (23, 'time'),
 (24, 'people')]

In [112]:
ops_field1 = {1: 21, 2: 22, 3: 24, 4: 23}

In [113]:
t = 0
for idx in trange(lens):
    idx1 = t + v_crt[idx]
    vs = new_ads_crt.iloc[t:idx1, :]
    if len(vs) == 1:
        vs.iloc[0, 1] = vs.iloc[0, 7]
        vs.loc[0, ops_field[vs.iloc[0, 9]]] = vs.iloc[0, 10]
    else:
        for i in range(1, len(vs)):
            vs.iloc[0, 1] = vs.iloc[i, 7]
            vs.iloc[0, ops_field1[vs.iloc[i, 9]]] = vs.iloc[i, 10]
    t = idx1
    ads_crt.iloc[idx] = vs.iloc[0]

  0%|          | 0/28685 [00:00<?, ?it/s]SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  2%|▏         | 488/28685 [01:39<1:36:32,  4.87it/s]SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 28685/28685 [1:46:58<00:00,  3.21it/s]


In [114]:
ads_crt.to_csv('../data/ads_crt.csv', index=None, encoding='utf-8')

In [46]:
ads_crt.shape

(28685, 25)

In [116]:
ads_crt.head().T

,0,1,2,3,4
广告id,31,32,84,85,128
创建时间,2019-03-12 23:22:50,2018-11-30 07:40:17,2019-03-06 23:52:47,2019-02-27 07:52:07,2019-03-06 21:40:09
广告账户id,12577,18752,789,6944,9895
商品id,18683,32534,18194,11250,31047
商品类型,13,13,13,13,13
广告行业id,224,136,98,170,17
素材尺寸,40,40,30,34,30
创建/修改时间,2019-03-12 23:22:50,2018-11-30 07:40:17,2019-03-06 23:52:47,2019-02-27 07:52:07,2019-03-06 21:40:09
操作类型,2,2,2,2,2
修改字段,2,4,4,2,2


In [118]:
ads_crt = pd.read_csv('../data/ads_crt.csv', encoding='utf-8')

- 注意备份

- 到目前为止的修改次数

In [120]:
ads_crt['cnt'] = pd.Series(v_crt)  

In [122]:
ads_crt.head().T

,0,1,2,3,4
广告id,31,32,84,85,128
创建时间,2019-03-12 23:22:50,2018-11-30 07:40:17,2019-03-06 23:52:47,2019-02-27 07:52:07,2019-03-06 21:40:09
广告账户id,12577,18752,789,6944,9895
商品id,18683,32534,18194,11250,31047
商品类型,13,13,13,13,13
广告行业id,224,136,98,170,17
素材尺寸,40,40,30,34,30
创建/修改时间,2019-03-12 23:22:50,2018-11-30 07:40:17,2019-03-06 23:52:47,2019-02-27 07:52:07,2019-03-06 21:40:09
操作类型,2,2,2,2,2
修改字段,2,4,4,2,2


In [125]:
ads_crt.drop(columns=['创建/修改时间', '操作类型', '修改字段', '操作后的字段值'], inplace=True)

In [126]:
ads_crt.to_csv('../data/ads_crt.csv', index=None, encoding='utf-8')